# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [15]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cities = pd.read_csv("../Output_data/cities.csv")
cities.head()

,City ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Kuandian,40.7286,124.7847,86.774,61,80,2.56,CN,1627101794
1,1,Etah,27.6333,78.6667,96.224,48,64,3.68,IN,1627101794
2,2,Belmonte,-15.8631,-38.8828,66.704,77,100,4.69,BR,1627101794
3,3,Kānker,20.2719,81.4931,74.246,92,100,4.27,IN,1627101795
4,4,Saint-Paul-lès-Dax,43.7272,-1.0516,62.672,97,10,1.03,FR,1627101795


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [47]:
gmaps.configure(api_key=g_key)
fig = gmaps.figure()
locations = cities[["Lat", "Lng"]].astype(float)
humidity = cities["Humidity"].astype(float)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [ ]:
A max temperature lower than 80 degrees but higher than 70.

Wind speed less than 10 mph.

Zero cloudiness.

Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.

Note: Feel free to adjust to your specifications but be sure to limit the number of rows returned by your API requests to a reasonable number.

In [8]:
### Weather Conditions ###
cities_narrow = cities.loc[(cities['Max Temp']<80)&(cities['Max Temp']>70)&(cities['Wind Speed']<10)&(cities['Cloudiness']==0)]
cities_narrow

,City ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
82,82,Byron Bay,-28.6500,153.6167,73.508,48,0,5.34,AU,1627101366
106,106,Bayir,37.2687,28.2167,73.922,45,0,2.15,TR,1627101823
136,136,Poum,-20.2333,164.0167,74.804,82,0,3.14,NC,1627101831
148,148,Genç,38.7507,40.5603,75.560,43,0,6.17,TR,1627101835
170,170,Novoil'insk,51.6829,108.7143,77.288,37,0,3.32,RU,1627101841
200,200,Avarua,-21.2078,-159.7750,75.254,73,0,3.60,CK,1627101447
225,225,Turaif,31.6725,38.6637,71.852,83,0,5.14,SA,1627101860
258,258,Krasnyye Barrikady,46.2045,47.8535,77.000,61,0,4.00,RU,1627101870
310,310,Abu Samrah,35.3029,37.1841,77.540,63,0,4.03,SY,1627101887
357,357,Saint-Pierre,-21.3393,55.4781,73.040,52,0,2.57,RE,1627101902


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [40]:
hotel_df = pd.DataFrame({})
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key,
}
for index,row in cities_narrow.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    hotel = requests.get(base_url, params=params).json()    
    try:
        hotel_df.loc[index,'Hotel Name'] = hotel["results"][0]["name"]
    except (KeyError, IndexError):
        hotel_df.loc[index,'Hotel Name'] = "No Hotel within 5000 meters"
hotel_df 

,Hotel Name
82,Backpackers Inn on the Beach
106,Hotel Babadag
136,Gîte kajeon
148,Genc Ogretmenevi
170,No Hotel within 5000 meters
200,Paradise Inn
225,Turaif
258,Baza Otdykha
310,No Hotel within 5000 meters
357,Lindsey Hôtel


In [44]:
hotels = pd.concat([cities_narrow,hotel_df],axis=1).reset_index()
hotels

,index,City ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,82,82,Byron Bay,-28.6500,153.6167,73.508,48,0,5.34,AU,1627101366,Backpackers Inn on the Beach
1,106,106,Bayir,37.2687,28.2167,73.922,45,0,2.15,TR,1627101823,Hotel Babadag
2,136,136,Poum,-20.2333,164.0167,74.804,82,0,3.14,NC,1627101831,Gîte kajeon
3,148,148,Genç,38.7507,40.5603,75.560,43,0,6.17,TR,1627101835,Genc Ogretmenevi
4,170,170,Novoil'insk,51.6829,108.7143,77.288,37,0,3.32,RU,1627101841,No Hotel within 5000 meters
5,200,200,Avarua,-21.2078,-159.7750,75.254,73,0,3.60,CK,1627101447,Paradise Inn
6,225,225,Turaif,31.6725,38.6637,71.852,83,0,5.14,SA,1627101860,Turaif
7,258,258,Krasnyye Barrikady,46.2045,47.8535,77.000,61,0,4.00,RU,1627101870,Baza Otdykha
8,310,310,Abu Samrah,35.3029,37.1841,77.540,63,0,4.03,SY,1627101887,No Hotel within 5000 meters
9,357,357,Saint-Pierre,-21.3393,55.4781,73.040,52,0,2.57,RE,1627101902,Lindsey Hôtel


In [50]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotels.iterrows()]
marker_locations = hotels[["Lat", "Lng"]]

In [51]:
# Add marker layer ontop of heat map
gmaps.configure(api_key=g_key)
fig = gmaps.figure()
locations = cities[["Lat", "Lng"]].astype(float)
humidity = cities["Humidity"].astype(float)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)
marker_layer = gmaps.marker_layer(marker_locations,
                                        info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))